In [60]:
from openai import OpenAI
import os
from elasticsearch import Elasticsearch
import json
import requests
from tqdm.auto import tqdm
import tiktoken

In [4]:
client=OpenAI()

### Q1

In [6]:
!curl localhost:9200

{
  "name" : "3d9c0716ed30",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "97nHsS8VQDKrjFivYErthg",
  "version" : {
    "number" : "9.0.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "73f7594ea00db50aa7e941e151a5b3985f01e364",
    "build_date" : "2025-04-30T10:07:41.393025990Z",
    "build_snapshot" : false,
    "lucene_version" : "10.1.0",
    "minimum_wire_compatibility_version" : "8.18.0",
    "minimum_index_compatibility_version" : "8.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [5]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2

In [11]:
es_client=Elasticsearch('http://localhost:9200')

In [15]:
index_settings=index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name='course-question-homework'

In [16]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-question-homework'})

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Q3

In [43]:
query="How do execute a command on a Kubernetes pod?"
def elastic_search(query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    response=es_client.search(index=index_name, body=search_query)
    results=[]
    for text in response["hits"]['hits']:
        results.append(text['_score'])
    return results
elastic_search(query)

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

### Q4

In [49]:
query="How do copy a file to a Docker container?"
def elastic_search(query):
    search_query={
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    'term':{
                        'course':'machine-learning-zoomcamp'
                    }
                }
            }
        }
    }
    response=es_client.search(index=index_name, body=search_query)
    results=[]
    for text in response["hits"]['hits']:
        results.append(text['_source'])
    return results
results=elastic_search(query)
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

**Answer:** 'How do I copy files from a different folder into docker container’s working directory?'

### Q5

In [57]:
context_template = """
Q: {question}
A: {text}
""".strip()
context=""
for doc in results:
    context+='\n\n'+context_template.format(question=doc['question'],text=doc['text']) if context else context_template.format(question=doc['question'],text=doc['text'])
    
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
prompt=prompt_template.format(question=query,context=context)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [58]:
len(prompt)

1446

### Q6

In [64]:
encoding = tiktoken.encoding_for_model("gpt-4o")
token_ids=encoding.encode(prompt)
len(token_ids)

320

**Answer:** 320

In [67]:
for token in token_ids:
    print(encoding.decode_single_token_bytes(token))

b"You're"
b' a'
b' course'
b' teaching'
b' assistant'
b'.'
b' Answer'
b' the'
b' QUESTION'
b' based'
b' on'
b' the'
b' CONT'
b'EXT'
b' from'
b' the'
b' FAQ'
b' database'
b'.\n'
b'Use'
b' only'
b' the'
b' facts'
b' from'
b' the'
b' CONT'
b'EXT'
b' when'
b' answering'
b' the'
b' QUESTION'
b'.\n\n'
b'QUESTION'
b':'
b' How'
b' do'
b' copy'
b' a'
b' file'
b' to'
b' a'
b' Docker'
b' container'
b'?\n\n'
b'CON'
b'TEXT'
b':\n'
b'Q'
b':'
b' How'
b' do'
b' I'
b' debug'
b' a'
b' docker'
b' container'
b'?\n'
b'A'
b':'
b' Launch'
b' the'
b' container'
b' image'
b' in'
b' interactive'
b' mode'
b' and'
b' overriding'
b' the'
b' entry'
b'point'
b','
b' so'
b' that'
b' it'
b' starts'
b' a'
b' bash'
b' command'
b'.\n'
b'docker'
b' run'
b' -'
b'it'
b' --'
b'entry'
b'point'
b' bash'
b' <'
b'image'
b'>\n'
b'If'
b' the'
b' container'
b' is'
b' already'
b' running'
b','
b' execute'
b' a'
b' command'
b' in'
b' the'
b' specific'
b' container'
b':\n'
b'docker'
b' ps'
b' ('
b'find'
b' the'
b' container'
b'-id'
b'

In [68]:
encoding.decode(token_ids)

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container

### Bonus: generating the answer

In [71]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user','content':prompt}]
)
response.choices[0].message.content

'You can copy a file to a Docker container using the `docker cp` command. The basic syntax to copy a file from your local machine to a running Docker container is:\n\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```\n\nThis command will transfer the specified file or directory from your local machine to the specified path inside the Docker container.'

In [75]:
len(response.choices[0].message.content)

385

### Bonus: calculating the costs 

In [73]:
output_tokens=150
input_tokens=250
requests=1000

In [74]:
single_cost=input_tokens*5/1000000+output_tokens*20/1000000
total_cost=requests*single_cost
total_cost

4.25

#### based on my values in Q6 and Q7

In [76]:
output_tokens=320
input_tokens=385
my_cost=input_tokens*5/1000000+output_tokens*20/1000000
my_cost

0.008325